In [36]:
# %pip install requests beautifulsoup4

#### 1. 초기 설정 및 라이브러리 임포트
이 셀에서는 필요한 라이브러리를 임포트하고, 스크래핑에 필요한 초기 변수들을 설정합니다.

In [1]:
# %%
import requests
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime
import time
import json # JSON 파일 처리를 위해 추가

# 스크래핑할 기본 URL 설정
BASE_URL_TEMPLATE = "https://business.nikkei.com/atcl/gen/19/00461/?TOC={page_num}"
# 초기 페이지 번호
initial_page_num = 1
# 초기 키워드 설정
initial_keyword = "business"
# 초기 ID 번호 (문자열 형식)
initial_id_num = "001"

# 저장할 디렉토리 설정: 코드가 위치한 루트 디렉토리의 'article_data' 폴더
output_dir = os.path.join(os.getcwd(), "article_data")

# 출력 디렉토리 생성 (없을 경우)
os.makedirs(output_dir, exist_ok=True)

# 봇으로 인식되지 않기 위한 User-Agent 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    'Referer': 'https://business.nikkei.com/' # Referer 추가: 스크래핑 시작점 도메인으로 설정
}

# 쿠키 파일 경로 (스크립트와 같은 디렉토리에 저장, 만료 시 다시 로그인해서 갱신 필요. ID와 비밀번호는 vesabeg992@datingso.com)
COOKIES_FILE = 'nikkei_cookies.json'

print(f"스크래핑 시작을 위한 초기 설정이 완료되었습니다.")
print(f"기본 URL 템플릿: {BASE_URL_TEMPLATE}")
print(f"초기 페이지 번호: {initial_page_num}")
print(f"초기 키워드: {initial_keyword}")
print(f"저장될 디렉토리: {output_dir}")
print(f"사용될 User-Agent: {headers['User-Agent']}")
print(f"쿠키 파일 경로: {COOKIES_FILE}")

# %%

스크래핑 시작을 위한 초기 설정이 완료되었습니다.
기본 URL 템플릿: https://business.nikkei.com/atcl/gen/19/00461/?TOC={page_num}
초기 페이지 번호: 1
초기 키워드: business
저장될 디렉토리: c:\sk\mini_project3\ai-service\data\article_data
사용될 User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36
쿠키 파일 경로: nikkei_cookies.json


#### 2. 페이지 URL에서 기사 링크 추출 함수
이 셀에서는 주어진 페이지 URL에서 모든 기사 링크를 추출하는 함수를 정의합니다.

In [38]:
# %%
# requests.Session() 객체를 전역으로 생성 (이전 셀에선 선언되지 않았으므로 이 위치에 추가)
session = requests.Session()

def get_article_links(page_url):
    """
    주어진 URL에서 기사 링크를 추출합니다.
    무료 기사 (class="p-articleList_item_date -titleLock -nbodd")만 필터링합니다.
    """
    print(f"\n페이지에서 기사 링크를 추출합니다: {page_url}")
    try:
        # headers를 요청에 포함하고 session 객체 사용
        response = session.get(page_url, headers=headers)
        response.raise_for_status()  # HTTP 오류 발생 시 예외 발생
    except requests.exceptions.RequestException as e:
        print(f"URL 접속 중 오류 발생: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # <div class="p-articleList_list"> 컨테이너 찾기
    article_list_container = soup.find('div', class_='p-articleList_list')
    
    if not article_list_container:
        print("기사 목록 컨테이너를 찾을 수 없습니다.")
        return []
    
    # 모든 <a class="p-articleList_item_link"> 태그 조회
    links = article_list_container.find_all('a', class_='p-articleList_item_link')
    
    article_urls = []
    for link in links:
        # link (<a> 태그) 내부에 무료 기사를 나타내는 <p> 태그가 있는지 확인
        # <p class="p-articleList_item_date -titleLock -nbodd">
        free_article_indicator = link.find('p', class_='p-articleList_item_date -titleLock -nbodd')
        
        if free_article_indicator: # 무료 기사 지표가 존재하는 경우에만 링크 추출
            href = link.get('href')
            if href:
                # 상대 경로를 절대 경로로 변환 (필요한 경우)
                if not href.startswith('http'):
                    href = f"https://business.nikkei.com{href}" 
                article_urls.append(href)
        else:
            # 유료 기사 링크는 건너뜀 (선택 사항: 디버깅을 위해 이 부분에 print를 넣을 수도 있음)
            # print(f"  유료 기사 링크 건너뜀: {link.get('href')}")
            pass
            
    print(f"총 {len(article_urls)}개의 무료 기사 링크를 찾았습니다.")
    return article_urls

# %%

#### 3. 단일 기사 페이지 스크래핑 및 저장 함수
이 셀에서는 각 기사 페이지로 이동하여 날짜, 본문 내용을 스크래핑하고 마크다운 파일로 저장하는 함수를 정의합니다.

In [39]:
# %%
def scrape_article_and_save(article_url, current_keyword, current_id_num):
    """
    단일 기사 페이지를 스크래핑하고 마크다운 파일로 저장합니다.
    """
    print(f"\n기사 스크래핑 중: {article_url}")
    try:
        # headers를 요청에 포함하고 session 객체 사용
        response = session.get(article_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"기사 페이지 접속 중 오류 발생: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # 기사 컨테이너 찾기
    article_container = soup.find('div', class_='articleBody p-article_body') 
    if not article_container:
        print("기사 컨테이너를 찾을 수 없습니다.")
        return None

    # 제목 스크래핑
    title_tag = article_container.find('p', class_='p-article_header_title')
    title = title_tag.get_text(strip=True) if title_tag else ""
    print(f"  제목: {title}\n") # 줄바꿈 추가

    # 작성 날짜 스크래핑
    datetime_str = "unknown_date"
    date_item = soup.find('li', class_='p-article_header_meta_item -date')
    if date_item:
        time_tag = date_item.find('time')
        if time_tag and 'datetime' in time_tag.attrs:
            datetime_str = time_tag['datetime']
            print(f"  작성 날짜: {datetime_str}")
        else:
            print("  작성 날짜 (time 태그 또는 datetime 속성)를 찾을 수 없습니다.")
    else:
        print("  작성 날짜 (li 태그)를 찾을 수 없습니다.")

    # 본문 내용 스크래핑
    paragraphs = article_container.find_all('p')
    article_text = []
    for p in paragraphs:
        cleaned_text = p.get_text(strip=True)
        if cleaned_text: 
            article_text.append(cleaned_text)
            
    full_article_content = "\n".join(article_text)
    
    # 파일 이름 생성
    file_date = datetime_str
    if re.match(r"\d{4}-\d{2}-\d{2}", datetime_str): 
        file_date = datetime_str
    elif "T" in datetime_str: 
        file_date = datetime_str.split('T')[0]
    
    filename = f"article_nikkei_{current_keyword}_{file_date}_{current_id_num}.md"
    filepath = os.path.join(output_dir, filename)

    # 마크다운 파일 내용 작성
    markdown_content = f"### {title}\n\n"
    markdown_content += full_article_content

    # 파일 저장
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        print(f"  기사 저장 완료: {filepath}")
        return True
    except IOError as e:
        print(f"파일 저장 중 오류 발생: {e}")
        return False

# %%

#### 4. 메인 스크래핑 루프
이 셀에서는 전체 스크래핑 과정을 제어하는 메인 루프를 실행합니다.

In [40]:
# %%

def load_cookies_into_session(session, cookie_file):
    """
    JSON 형식의 쿠키 파일을 읽어 requests.Session 객체에 로드합니다.
    """
    if not os.path.exists(cookie_file):
        print(f"오류: 쿠키 파일 '{cookie_file}'을 찾을 수 없습니다.")
        print("브라우저에서 수동으로 로그인 후 'EditThisCookie' 등의 확장 프로그램으로 쿠키를 JSON으로 내보내고 해당 파일을 스크립트와 같은 위치에 저장해주세요.")
        return False

    try:
        with open(cookie_file, 'r', encoding='utf-8') as f:
            cookies_data = json.load(f)
        
        for cookie in cookies_data:
            # domain을 requests.cookies.RequestsCookieJar에 맞는 형식으로 처리
            # .으로 시작하는 경우 제거 (requests는 자동으로 처리하지만 명시적으로 제거하여 혹시 모를 문제 방지)
            domain = cookie.get('domain', '').lstrip('.') 
            
            # expirationDate가 존재하고 유효한 경우만 expires 설정
            expires_at = cookie.get('expirationDate')
            if expires_at:
                try:
                    # JavaScript timestamp는 초 단위, Python time.time()도 초 단위
                    expires_at = int(expires_at)
                except (ValueError, TypeError):
                    expires_at = None # 유효하지 않은 값은 None으로 처리

            session.cookies.set(
                cookie['name'],
                cookie['value'],
                domain=domain,
                path=cookie.get('path', '/'),
                expires=expires_at, # expires에 None 또는 유효한 timestamp 전달
                secure=cookie.get('secure', False),
                rest={'HttpOnly': cookie.get('httpOnly', False)}
            )
        print(f"'{cookie_file}'에서 {len(cookies_data)}개의 쿠키를 세션에 성공적으로 로드했습니다.")
        return True
    except json.JSONDecodeError as e:
        print(f"오류: 쿠키 파일 '{cookie_file}'이 유효한 JSON 형식이 아닙니다: {e}")
        return False
    except Exception as e:
        print(f"쿠키 로드 중 알 수 없는 오류 발생: {e}")
        return False

# 메인 스크래핑 루프
current_page_num = initial_page_num
current_keyword = initial_keyword
current_id_int = int(initial_id_num) 

# 최대 스크래핑 페이지 수 설정 (선택 사항)
max_pages_to_scrape = 20

# 쿠키 로드 시도
if not load_cookies_into_session(session, COOKIES_FILE):
    print("쿠키 로드에 실패하여 스크래핑을 시작할 수 없습니다. 쿠키 파일을 확인해주세요.")
else:
    print("쿠키 로드 성공! 로그인된 상태로 기사 스크래핑을 시작합니다.")
    
    # 로그인 상태 확인 (선택 사항: 실제로 로그인되었는지 확인하기 위한 요청)
    try:
        # business.nikkei.com의 메인 페이지 또는 로그인 후에만 접근 가능한 페이지로 테스트
        test_response = session.get("https://business.nikkei.com/", headers=headers, allow_redirects=True)
        test_response.raise_for_status()
        
        # 'ログイン' (로그인) 텍스트가 페이지에 있고, 'ログアウト' (로그아웃) 텍스트가 없으면 로그인 안된 걸로 간주
        # (이 부분은 웹사이트의 실제 로그인/로그아웃 텍스트에 따라 다를 수 있습니다.)
        if "ログイン" in test_response.text and "ログアウト" not in test_response.text:
             print("경고: 쿠키 로드 후에도 로그인 상태가 아닌 것으로 보입니다. 쿠키가 만료되었거나 올바르지 않을 수 있습니다.")
             print("스크래핑이 예상대로 작동하지 않을 수 있습니다.")
        else:
            print("세션이 로그인 상태인 것으로 보입니다. 스크래핑을 진행합니다.")
    except requests.exceptions.RequestException as e:
        print(f"로그인 상태 확인 중 오류 발생: {e}. 스크래핑을 진행합니다.")

    while True:
        page_url = BASE_URL_TEMPLATE.format(page_num=current_page_num)
        print(f"\n--- 현재 페이지: {page_url} (페이지 번호: {current_page_num}) ---\n") # 줄바꿈 추가

        article_links = get_article_links(page_url)

        if not article_links:
            print(f"더 이상 기사 링크를 찾을 수 없거나 페이지가 존재하지 않습니다. 스크래핑을 종료합니다.")
            break

        for link in article_links:
            formatted_id_num = f"{current_id_int:03d}"
            
            success = scrape_article_and_save(link, current_keyword, formatted_id_num)
            if success:
                current_id_int += 1 
            else:
                print(f"경고: {link} 스크래핑에 실패했습니다. 다음 기사로 넘어갑니다.")
            
            time.sleep(1) 

        print(f"\n페이지 {current_page_num}의 모든 기사 스크래핑을 완료했습니다.")
        
        current_page_num += 1
        
        if 'max_pages_to_scrape' in locals() and current_page_num > initial_page_num + max_pages_to_scrape -1 :
            print(f"설정된 최대 스크래핑 페이지 수({max_pages_to_scrape} 페이지)에 도달했습니다. 스크래핑을 종료합니다.")
            break

        time.sleep(2)

print("\n--- 모든 스크래핑 작업이 완료되었습니다. ---\n")

'nikkei_cookies.json'에서 33개의 쿠키를 세션에 성공적으로 로드했습니다.
쿠키 로드 성공! 로그인된 상태로 기사 스크래핑을 시작합니다.
세션이 로그인 상태인 것으로 보입니다. 스크래핑을 진행합니다.

--- 현재 페이지: https://business.nikkei.com/atcl/gen/19/00109/?TOC=10 (페이지 번호: 10) ---


페이지에서 기사 링크를 추출합니다: https://business.nikkei.com/atcl/gen/19/00109/?TOC=10
총 0개의 무료 기사 링크를 찾았습니다.
더 이상 기사 링크를 찾을 수 없거나 페이지가 존재하지 않습니다. 스크래핑을 종료합니다.

--- 모든 스크래핑 작업이 완료되었습니다. ---

